In [ ]:
import cv2 # for capturing video using camera

In [ ]:
import os # for using directory navigations

In [ ]:
import face_recognition as fr # used for face recognition modules

In [ ]:
import numpy as npy #used for organising the data in arrays

In [ ]:
import matplotlib.pyplot as plt # for displaying the data collected using camera

In [ ]:
#this xml file was taken from # https://github.com/kipr/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml

In [ ]:
traindata = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
# user-defined function to apply the image resizing on the frames that are captured

def filteredloop(loop_data,tempdata):
    for x,y,w,h in loop_data:# x,y cordinates and width and height of the image
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,255), 4) # to display rectangle over the identified image that takes x,y cordinates and width and height as parameters
        updated_frame = frame[y:y+h, x:x+w, :]# rows and columns of the face detected in the frame
        updated_frame = cv2.resize(updated_frame, (50,50))#resizing all the identified faces in the frame to standard 50*50 size
        print(len(tempdata))
        if len(tempdata) < 200:#loop for capturing 200 frames and storing them
             tempdata.append(updated_frame)
    
    

In [ ]:
without_mask_data = [] # to store the data of the images without having mask
capture = cv2.VideoCapture(0)#to turn on the camera module
while True:
    #isflagTrue contains true or false. read() returns true to this variable if the camera module is working fine
    
    isflagTrue , frame = capture.read()#to input the frames from the camera module
    if(isflagTrue):
        currentframe = traindata.detectMultiScale(frame)#perfroms sliding window operation and detects the haar features in every window. This line detects the faces in the current image frame
        filteredloop(currentframe,without_mask_data)
        cv2.imshow('result',frame)
        # ascii value of letter "q" is 2. if user clicks "q", the loop will be terminated
        if cv2.waitKey(2) == 113 or len(without_mask_data) >= 200:
            break
    
capture.release()# release the camera module
cv2.destroyAllWindows() # to close the popup camera window



In [ ]:
npy.save('nomask.npy',without_mask_data)# saving the results in a file

In [ ]:
#Module to capture data with mask

mask_data = [] # to store the data of the images having mask
capture = cv2.VideoCapture(0) #to turn on the camera module
while True:
    isflagTrue , frame = capture.read() #isflagTrue contains true or false. read() returns true to this variable if the camera module is working fine
    if(isflagTrue):
        currentframe = traindata.detectMultiScale(frame)
        filteredloop(currentframe,mask_data)
        cv2.imshow('result',frame)
        if cv2.waitKey(2) == 113 or len(mask_data) >= 200:#the loop terminates when user clicks "q" or when it completes taking 200 pics
            break
    
capture.release()
cv2.destroyAllWindows()



In [ ]:
npy.save('mask.npy',mask_data)#saving the details in a file

In [ ]:
plt.imshow(mask_data[116]) # to show a random file in the data captured

In [ ]:
#loading the data from the files
mask = npy.load('mask.npy')
nomask = npy.load('nomask.npy')

In [ ]:
#function to convert the data in the files into 2D images
def reshape(data):
    return data.reshape(200,50 * 50 *3)

In [ ]:
#converting the data using the user-defined function
mask = reshape(mask)
nomask = reshape(nomask)

In [ ]:
#joining the data together to a single dataset using concatination
combined_data = npy.r_[mask, nomask]

In [ ]:
#assigning the first 200 dataset values in the combined dataset to 0 and the rest 200 values to 1
indexes = npy.zeros(combined_data.shape[0])
indexes[200:] = 1.0

In [ ]:
#https://scikit-learn.org/stable/
#https://scikit-learn.org/stable/modules/svm.html

In [ ]:
from sklearn.svm import SVC #to load the linear support vector classifier from sklearn
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split # predefined function to split the data into training and testing data

In [ ]:
x1, x2, y1, y2 = train_test_split(combined_data,indexes, test_size=0.25)#To prevent overfitting

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=3) #principal component analysis to reduce the linear dimensity  


In [ ]:
x1 = pca.fit_transform(x1)

In [ ]:
x1, x2, y1, y2 = train_test_split(combined_data,indexes, test_size=0.25)

In [ ]:
svm = SVC()
svm.fit(x1, y1)

In [ ]:
predictedvalue = svm.predict(x2)

In [ ]:
accuracy_score(y2, predictedvalue) #getting the accuracy over the test data

In [ ]:
#to display the values Mask and No mask on the rectangle over the image
values = {0 : 'Mask', 1 : 'No Mask'}
f_style = cv2.FONT_HERSHEY_COMPLEX

In [ ]:
# user-defined function to resize the images to standard 50*50
def adjustframe(frame):
    adjustedframe = frame[y:y+h, x:x+w, :]
    adjustedframe = cv2.resize(adjustedframe, (50,50))
    adjustedframe = adjustedframe.reshape(1,-1)
    return svm.predict(adjustedframe)[0]

In [ ]:
#user defined function for loading images from designated Directory
images = []#array for storing the images
c_names = []#array for storing the names of the images
myList = os.listdir("ai_testimages")#loading the names of the images
myList.remove(".ipynb_checkpoints")#removing the temp file that was created by default by Jupyter Notebook
print(myList)
#loading the images from the designated directory to the images array
for i in myList:
    currentImg = cv2.imread(f'{"ai_testimages"}/{i}')
    images.append(currentImg)
    c_names.append(os.path.splitext(i)[0])#saving the names of the image files
print(c_names)

In [ ]:
#encoding the images from the designated path
f_encode = []
img_encoding = []
for i in myList:
    Img = fr.load_image_file(f'{"ai_testimages"}/{i}')
    img_encoding.append(fr.face_encodings(Img)[0])
print(img_encoding)


In [ ]:
#variable initialisation 
f_loc = []
f_encode = []
f_names = []
p_frame = True

In [ ]:
# user defined function for encoding the images from the designated directory
def encode(f_encode):
    for i in f_encode:
        cr_match = fr.compare_faces(img_encoding, i)
        name = "Unknown"

        face_dist = fr.face_distance(img_encoding, i)
        b_m_i = npy.argmin(face_dist)
        if cr_match[b_m_i]:
            name = c_names[b_m_i]

        f_names.append(name)

In [ ]:
#user defined function to display the result
def result(cresult):
     for (t, r, b, l), name in cresult:
        t *= 4
        r *= 4
        b *= 4
        l *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (l, t), (r, b), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (l, b - 35), (r, b), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_COMPLEX
        cv2.putText(frame, name, (l + 6, b - 6), font, 1.0, (255, 255, 255), 1)
        cv2.putText(frame, result1, (l + 6, t - 6), font, 1.0, (255, 255, 255), 1)


In [ ]:
#module to start the live detection
traindata = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
capture = cv2.VideoCapture(0) #for capturing the video frame
video_capture = cv2.VideoCapture(0)
data = []
result1 =""
cresult =""
while(1):
    isFlagTrue, frame = capture.read()
    if(isFlagTrue):
        currentframe = traindata.detectMultiScale(frame)
        for x,y,w,h in currentframe:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,255), 4)# to display rectangle over the identified image that takes x,y cordinates and width and height as parameters
            result1 = values[int(adjustframe(frame))]# converting the 
            #cv2.putText(frame, result, (x,y), f_style, 1, (244,250,250), 2) #to show the results over the rectangle "Mask", "No Mask"
            print(result1) # to print the result
        # Removed as a part of phase 2
        #cv2.imshow('result',frame)
        #result1=result
        #if cv2.waitKey(33) & 0xFF == ord('q'):## to terminate the loop when the user clicks "q"
         #   break

 
    #phase II:
    ret, frame = video_capture.read() #to capture a frame

    # Resizing the image 
    s_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Converting the video frame to standard input image used by face recognition
    r_frame = s_frame[:, :, ::-1]
    
    if p_frame:
        f_loc = fr.face_locations(r_frame)
        f_encode = fr.face_encodings(r_frame, f_loc)
        f_names = []
        encode(f_encode)     
    p_frame = not p_frame
    # Display the results
    cresult = zip(f_loc, f_names)
    
    result(cresult)
    # Display the resulting image
    cv2.imshow('Video',frame)
    #cv2.imshow('result1',frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()



